In [ ]:
using Revise
using RDDIP
using Random
using Plots
using Gurobi
using Statistics
const GRB_ENV = Gurobi.Env()
optimizer=() -> Gurobi.Optimizer(GRB_ENV)

In [ ]:
T = 5
graph=RDDIP.LinearGraph(T);

In [ ]:
function subproblem_builder(instance::RDDIP.Instance, force::Float64, subproblem::Model, node::Int)
    # State variables
    H = [0.0, 50.0, 100.0, 150.0, 200.0]
    V = [0 for i in 1:5]
    V[5] = 1
    @variable(subproblem, volume[i in 1:5], Bin, RDDIP.State, initial_value = V[i])
    # @variable(subproblem, is_on, Bin, RDDIP.State, initial_value = 1)
    # Control variables
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation >= 0
        hydro_spill >= 0
    end)
    # Random variables
    @variable(subproblem, inflow)

    K=9
    Ω = [[k*200/K for k in 1:K] for t in 1:T]
    P = [[1/K for k in 1:K] for t in 1:T]
    Ω[1] = [50.0]
    P[1] = [1.0]
    RDDIP.parameterize(subproblem, Ω[node], P[node]) do ω
        return JuMP.fix(inflow, ω)
    end
    # Transition function and constraints
    @constraints(
        subproblem,
        begin
            sum(H[i] * volume[i].out for i in 1:5) == sum(H[i] * volume[i].in for i in 1:5) - hydro_generation - hydro_spill + inflow
            # thermal_generation <= 150 * is_on.out
            sum(volume[i].out for i in 1:5) == 1
            demand_constraint, hydro_generation + thermal_generation == 150
        end
    )
    # Stage-objective
    @stageobjective(subproblem, 50 * thermal_generation + 60 * hydro_spill)
    return subproblem
end

In [ ]:
model = RDDIP.PolicyGraph(
    subproblem_builder,
    instance,
    0.0,
    graph;
    sense = :Min,
    lower_bound = 0.0,
    optimizer = optimizer,
)

In [ ]:
# println(model[1].subproblem)
model[2].noise_terms

In [ ]:
# RDDIP.train(model; iteration_limit = 5, duality_handler = RDDIP.ContinuousConicDuality())
RDDIP.train(model; iteration_limit = 100, duality_handler = RDDIP.LagrangianDuality())
# RDDIP.train(model; iteration_limit = 100, duality_handler = RDDIP.StrengthenedConicDuality())

In [ ]:
instance=RDDIP.parse_nc4("Data/T-Ramp/10_0_1_w.nc4",  optimizer, 24); instance.N

In [ ]:
res=RDDIP.bin_extensive_neutral_integer(instance; K=4, silent=false,  Tmax=2, force=0.0, S=1, batch=1, gap=0.001, timelimit=10);

In [ ]:
RDDIP.bin_extensive_neutral(instance; silent=false,  force=1.0, S=20, batch=1, gap=0.001, timelimit=10);

In [ ]:
resSP=RDDIP.benders_callback(instance, RDDIP.extended_BD_FH_OH, silent=false, force=1.0, S=20, batch=1, gap=0.1, timelimit=120);

In [ ]:
function subproblem_builder_UC(instance::Instance, force::Float64, subproblem::Model, node::Int)
    # State variables
    N=instance.N
    thermal_units=values(instance.Thermalunits)
    Next=instance.Next
    Buses=1:size(Next)[1]
    Lines=values(instance.Lines)
    BusWind=instance.BusWind
    NumWindfarms=length(BusWind)
    @variable(subproblem, thermal_units[i].MinPower <= power[i = 1:N] <= thermal_units[i].MaxPower, RDDIP.State, initial_value = thermal_units[i].MinPower)
    # Control variables
    @variables(subproblem, begin
        power_shedding >= 0
        power_curtailement >= 0
        is_on[i = 1:N, k = 1:2] >= 0 # k=1 current time, k=2 previous time
        start_up[i = 1:N] >= 0
        shut_down[i = 1:N] >= 0
        θ[b in Buses] 
        flow[b in Buses, bp in Next[b]]
    end)

    @constraint(model, [line in Lines, t in 1:T, s in 1:S], flow[line.b1,line.b2,t,s]<=line.Fmax)
    @constraint(model, [line in Lines, t in 1:T, s in 1:S], flow[line.b1,line.b2,t,s]>=-line.Fmax)
    @constraint(model, [line in Lines, t in 1:T, s in 1:S], flow[line.b1,line.b2,t,s]==line.B12*(θ[line.b1,t,s]-θ[line.b2,t,s]))

    # Random variables
    @variable(subproblem, error_forecast[b in Buses])
    Ω = [[0.0 for b in Buses]]
    P = [1.0]
    RDDIP.parameterize(subproblem, Ω, P) do ω
        for b in Buses
            JuMP.fix(error_forecast[b], ω[b])
        end
    end
    # Transition function and constraints
    @constraints(
        subproblem,
        begin
            [i = 1: N], power[i].out <= thermal_units[i].MaxPower * is_on[i, 1]
            [i = 1: N], power[i].out >= thermal_units[i].MinPower * is_on[i, 1]
            [i = 1: N], power[i].out - power[i].in <= (-thermal_units[i].DeltaRampUp)*start_up[i] + (thermal_units[i].MinPower + thermal_units[i].DeltaRampUp)*is_on[i, 1] - (thermal_units[i].MinPower)*is_on[i, 2]
            [i = 1: N], power[i].in - power[i].out <= (-thermal_units[i].DeltaRampDown)*shut_down[i] + (thermal_units[i].MinPower + thermal_units[i].DeltaRampDown)*is_on[i, 2] + (thermal_units[i].MinPower)*is_on[i, 1]
            [line in Lines], flow[line.b1,line.b2] <= line.Fmax
            [line in Lines], flow[line.b1,line.b2] >= -line.Fmax
            [line in Lines], flow[line.b1,line.b2] == line.B12*(θ[line.b1]-θ[line.b2])
            demand_constraint, [b in Buses], sum(power[unit.name].out for unit in thermal_units if unit.Bus==b) - power_curtailement[b] + power_shedding[b] == instance.Demandbus[b](1+force*error_forecast[b])+sum(flow[b,bp] for bp in Next[b])
        end
    )
    # Stage-objective
    @stageobjective(subproblem, sum(unit.LinearTerm*power[unit.name].out for unit in thermal_units)+sum(SHEDDING_COST*power_shedding[b]+CURTAILEMENT_COST*power_curtailement[b] for b in Buses))
    return subproblem
end

In [ ]:
T = 24
graph=RDDIP.LinearGraph(T);

In [71]:
force = 0.0
model = RDDIP.PolicyGraph(
    RDDIP.subproblem_builder_UC,
    instance,
    force,
    graph,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = optimizer,
)

A policy graph with 24 nodes.
 Node indices: 1, ..., 24


In [ ]:
# println(JuMP.upper_bound(collect(values(model[2].states))[1].in))
# println(model[2].lagrangian)
cut = model[2].lagrangian.constraints
# cut.outgoing_state_values[Symbol("power_integer[7,1]")]

In [74]:
model[23].noise_terms

2-element Vector{RDDIP.Noise}:
 RDDIP.Noise{Vector{Float64}}([0.0], 0.5)
 RDDIP.Noise{Vector{Float64}}([1.0], 0.5)

In [ ]:
RDDIP.train(model; iteration_limit = 2, duality_handler = RDDIP.StrengthenedConicDuality())
# RDDIP.train(model; iteration_limit = 1, duality_handler = RDDIP.LagrangianConicDuality())

# RDDIP.train(model; iteration_limit = 1, duality_handler = RDDIP.LagrangianDuality())

-------------------------------------------------------------------
         RDDIP.jl (c) Oscar Dowson and contributors, 2017-25
-------------------------------------------------------------------
problem
  nodes           : 24
  state variables : 50
  scenarios       : 8.38861e+06
  existing cuts   : true
options
  solver          : serial mode
  risk measure    : RDDIP.Expectation()
  sampling scheme : RDDIP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [165, 165]
  AffExpr in MOI.EqualTo{Float64}         : [61, 61]
  AffExpr in MOI.GreaterThan{Float64}     : [31, 82]
  AffExpr in MOI.LessThan{Float64}        : [70, 120]
  VariableRef in MOI.EqualTo{Float64}     : [51, 51]
  VariableRef in MOI.GreaterThan{Float64} : [33, 33]
  VariableRef in MOI.LessThan{Float64}    : [1, 1]
  VariableRef in MOI.ZeroOne              : [70, 70]
numerical stability report
  matrix range     [1e+00, 3e+04]
  objective range  [1e+00, 3e+03]
  bounds range     [0e+00,

┌ Warning: Re-training a model with existing cuts!
│ 
│ Are you sure you want to do this? The output from this training may be
│ misleading because the policy is already partially trained.
│ 
│ If you meant to train a new policy with different settings, you must
│ build a new model.
│ 
│ If you meant to refine a previously trained policy, turn off this
│ warning by passing `add_to_existing_cuts = true` as a keyword argument
│ to `RDDIP.train`.
│ 
│ In a future release, this warning may turn into an error.
└ @ RDDIP /home/mathis/Documents/RDDIP/src/algorithm.jl:1196


        1S  1.879157e+06  1.631773e+06  1.020649e+00       213   1
Lower bound: 1.6317730597142896e6
Lower bound: 1.6435738021926088e6
         3S  1.873362e+06  1.678899e+06  2.919240e+00       355   1
Lower bound: 1.6788990491572625e6
         4S  1.908575e+06  1.685827e+06  3.983234e+00       426   1
Lower bound: 1.6858265869598624e6
Lower bound: 1.7055454845723435e6
         6S  1.912767e+06  1.712381e+06  5.873501e+00       568   1
Lower bound: 1.7123811764952887e6
         7S  1.893347e+06  1.734472e+06  6.929450e+00       639   1
Lower bound: 1.7342999443226245e6
Lower bound: 1.7379300117629888e6
         9S  1.890797e+06  1.741797e+06  8.949249e+00       781   1
Lower bound: 1.7417965540989225e6
Lower bound: 1.749518200048355e6
Lower bound: 1.7501350370281113e6
Lower bound: 1.7656502578616655e6
Lower bound: 1.7660776361733507e6
        14S  1.849416e+06  1.767236e+06  1.439519e+01      1136   1
Lower bound: 1.7672363229656934e6
Lower bound: 1.7676069247077736e6
Lower bound: 1.7

In [ ]:
res=RDDIP.train(model; iteration_limit = 2, print_level = 0, duality_handler = RDDIP.RDDIP.LagrangianConicDuality())

In [ ]:
string([res.log[i].simulation_value for i in 1:length(res.log)])

In [ ]:
println(model[23].subproblem)

In [ ]:
println(model[23].subproblem)
# mod = model[23].subproblem
# optimize!(mod)

In [ ]:
node = model[23]
for (key, state) in node.states
    # println((key, value(state.in), JuMP.is_integer(state.in), JuMP.upper_bound(state.in)))
    println((key, state.out, JuMP.is_binary(state.out), node.incoming_state_bounds[key]))
    # println((key, value(state.in), JuMP.is_integer(state.in)))
    # println(JuMP.upper_bound(state.in))
end
JuMP.upper_bound(collect(values(node.states))[1].in)
# collect(values(node.states))[1].in


In [ ]:
println(model[1].subproblem)

In [ ]:
1.806476027737761e6, 1.8289582420327545e6

In [ ]:
mod = JuMP.Model(optimizer)
@variable(mod, x, Bin)
@variable(mod, y )
@objective(mod, Max, x)
# JuMP.optimize!(mod)
# JuMP.termination_status(mod)
println(mod)
undo_relax = JuMP.relax_integrality(mod)
println(mod)
# undo_relax()
JuMP.upper_bound(x)

In [ ]:
mod = model[2].lagrangian.model
# set_optimizer(mod, optimizer)
set_optimizer_attribute(mod, "Presolve", 0)
println(mod)
unset_silent(mod)  # Remet les sorties du solveur
optimize!(mod)
println("Termination status: ", JuMP.termination_status(mod))
println("Primal status:     ", JuMP.primal_status(mod))
Dict(i =>value.(var) for (i, var) in model[2].lagrangian.dual_variables)
for (i, var) in model[2].lagrangian.dual_variables
    if abs(value(var)) > 1e-6
        println("Variable ", i, " has value ", value(var))
    end
end
println(value(model[2].lagrangian.theta))